In [1]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 864.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

# 모델 로딩
프리트레인한 GPT2 모델과 토크나이저를 읽어 들입니다.

In [1]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [2]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## 프롬프트 준비

언어모델에 넣을 프롬프트를 준비합니다.

In [3]:
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

## Greedy Search


다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴합니다.
여러 번 수행하더라도 생성 결과가 바뀌지 않습니다 (`do_sample=False`).
`max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단합니다. `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제합니다.

In [4]:
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## Beam Search

Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성합니다. Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있습니다.

In [5]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
      num_beams=3,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


`num_beams=1`로 설정하면 정확히 Greedy search와 동일하게 작동합니다.


In [6]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 반복 줄이기

### 반복되는 n-gram 사이즈를 지정하기

위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됩니다. 이를 아래와 같이 지정해 반복을 방지합니다. 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.


In [7]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


### repetition penalty

repetition penalty로 반복을 통제할 수도 있습니다. 다음과 같이 실행하면 되며 그 범위는 1 이상의 값을 가져야 합니다. 1이라면 아무런 패널티를 적용하지 않는게 됩니다.


In [8]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



`repetition_penalty` 값이 클 수록 패널티가 세게 적용됩니다.

In [9]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [10]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [11]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


## top-k sampling

지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식을 살펴봤습니다. top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라집니다. k는 1 이상의 값을 지녀야 합니다.

In [12]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요'라는 문구를 넣은 문구를 넣은 경우도 있었습니다.
김 의원은 선거사무소 개소식이 열리던 전날 오전에도 이 문구를 붙인 사실이 드러나 논란이 일었습니다.
하지만 정 씨는 지난 11일 한 인터뷰에서 "선거사무소 개


k=1일 경우 Greedy search와 동일합니다.

In [13]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## top-k sampling + temperature scaling

top-k sampling은 temperature scaling과 동시에 적용할 수 있습니다. 그 값에 따라 다음과 같은 효과가 납니다.

(1) t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다

In [14]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



(2) t=1이라면 모델 출력 분포를 그대로 사용한다 > 하지만 샘플링 방식을 사용하기 때문에 생성할 때마다 다른 문장이 나온다

In [15]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요!`라는 글귀를 읽어보았지만, 그냥 안 읽어봤다는 사람이 한둘이 아닌 것 같습니다.
다른 사람은 안 읽어본다고 해서 알쏭달쏭한 내용도 있지만, 왜 지금 이 글을 읽고 있는지 이해가 안 된다고 느꼈


(3) t를 키울수록 토큰 분포가 uniform해진다 > 사실상 uniform sampling이 된다, 생성 품질이 악화할 가능성이 높아진다

In [16]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요</d> 박철준 의원이 그간의 청신기를 말해주기 싫지만, 또 하나의 건 아닌 만큼 뭇 남성의 목발을 찬다면 목욕으로 전환하라는 청원은 어떻게 됐습니다, 가수 문병완 역시. 네 가수도


## top-p sampling

top-p sampling은 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라집니다. p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라집니다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높입니다.

In [17]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요~?
저희는 올해도 열심히 준비했습니다.
내년에는 좀 더 열심히 하겠습니다.
저는 이번에 새로 뽑힌 이~ 장관은 누구보다도 능력있고 리더십이 있는 사람이에요.
그리고 무엇보다도 이번에 장관직을 수행하는 여러 가지 이유


p가 0에 가까울 경우 Greedy search와 비슷해 집니다.

In [18]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



## 통합 적용

저희가 실습에 사용하고 있는 허깅페이스(huggingface) 라이브러리의 구현상 적용 순서는 다음과 같습니다.

- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits

유효한 설정들을 종합 적용해 문장을 생성하는 코드는 다음과 같습니다. 샘플링(top-k, top-p)을 적용하기 때문에 시행 때마다 다른 문장이 생성됩니다.

In [19]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요~
아니 저는 원래 이렇게 예쁜걸 좋아해요..
저는 아직은 잘생겼는데 이제 삐끗하나봐요?
내가 다 커서 그런가??
어머. 뭐예요,
뭐!!
근데 그
